In [3]:
import pandas as pd
from pymongo import MongoClient
import numpy as np
import pandas as pd
import csv

import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag

file_names = []
cluster_num = 0 # Change this to match cluster
for i in range(9):
    file_names.append(f'cluster_{cluster_num}_{i}.txt')

text = ''
# Load the text file
for name in file_names:
    # For each file, open it in read mode
    with open(name, 'r', encoding="utf-8") as file:
        # Read its contents and append to the text string
        text += file.read() + '\n'

sentences = text.split("\n")

additional_stopwords = []
with open('added_stopwords.txt', 'r', encoding="utf-8") as file:
    additional_stopwords = [line.strip() for line in file]

# Define a function for custom preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = text.split()
    # Filter out stop words
    stop_words = set(stopwords.words('english')).union(additional_stopwords)
    # Filter out stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Get POS tags for filtered tokens
    tagged_tokens = pos_tag(filtered_tokens)
    # Filter out only nouns
    nouns = [token for token, pos in tagged_tokens if pos.startswith('N')]
    # Join tokens back into a single string
    preprocessed_text = ' '.join(nouns)
    return preprocessed_text

# Preprocess each sentence in the list
preprocessed_sentences = [preprocess_text(sentence) for sentence in sentences]

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the preprocessed sentences
vectorizer.fit(preprocessed_sentences)

# Transform the sentences into TF-IDF vectors
tfidf_vectors = vectorizer.transform(preprocessed_sentences)

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Calculate the average TF-IDF score for each word across all sentences
average_tfidf_scores = tfidf_vectors.mean(axis=0)

# Convert average TF-IDF scores to a dictionary with feature names as keys
word_tfidf_dict = dict(zip(feature_names, average_tfidf_scores.tolist()[0]))

# Sort the dictionary by TF-IDF score in descending order
sorted_word_tfidf = sorted(word_tfidf_dict.items(), key=lambda x: x[1], reverse=True)
    
# Print the top N words with highest average TF-IDF scores
top_n = 30 # Adjust the number of top words to display
print("Top words with highest average TF-IDF scores:")
for word, score in sorted_word_tfidf[:top_n]:
    print(f"Word: {word}, Average TF-IDF Score: {score}")


Top words with highest average TF-IDF scores:
Word: need, Average TF-IDF Score: 0.0014752111772180932
Word: students, Average TF-IDF Score: 0.001375062831720171
Word: school, Average TF-IDF Score: 0.0013743867915714653
Word: talk, Average TF-IDF Score: 0.0013564229557085266
Word: hours, Average TF-IDF Score: 0.0013250935102476298
Word: watch, Average TF-IDF Score: 0.0013234540355467723
Word: whats, Average TF-IDF Score: 0.0013159162675803228
Word: everything, Average TF-IDF Score: 0.0013060607460929278
Word: retweet, Average TF-IDF Score: 0.0013033056575228559
Word: health, Average TF-IDF Score: 0.0012971842166679555
Word: hope, Average TF-IDF Score: 0.0012904563279874783
Word: family, Average TF-IDF Score: 0.001283820693426421
Word: show, Average TF-IDF Score: 0.0012706558551509838
Word: post, Average TF-IDF Score: 0.001258386288053139
Word: change, Average TF-IDF Score: 0.0012565584152170778
Word: night, Average TF-IDF Score: 0.0012491584578582854
Word: story, Average TF-IDF Score: 0

## Analysis of Clusters
### Cluster 0: 
Political: 
Top words included 'congress', 'pm', 'country', 'president', 'india', 'parkistan', 'nation' indicate that this cluster is mostly related to political matters, primarily those in Asia.

### Cluster 1: 
Business:
Words like 'work', 'team', 'support', 'business', 'project', 'job' are common words that are used in tweets that pertain to general business.

Academic:
Words like 'students', 'research', 'paper', 'science', 'article', 'event' tend to be related to the field of academics.

### Cluster 2: 
Technology and Innovation:
Words like "technology", "ai" , "innovation", "python", "crypto", "industry", "tomorrow", "datascience" and "research" focus on news, insights or developments in technology and similar fields

### Cluster 3: 
Western Politics: 
Words like "war", "country", "russia", "president", "countries", "peace", "conference" suggest strong focus on geopolitical issues, particularly in a Western context. 

Public Health and Policies:
Words like "health", "treatment", and "covid19" suggest these political bots may cover political topics related to public health policies and issues

### Cluster 4: 
Everyday Topics:
Words like 'watch', 'health', 'family', 'tomorrow', 'fun', 'night' tend to be common words that are used in everyday conversations.